# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline


In [2]:
train = pd.read_csv("E:/Kaggle.Challenges/Machine.Learning.Titanic/Dataset/train.csv")
test = pd.read_csv("E:/Kaggle.Challenges/Machine.Learning.Titanic/Dataset/test.csv")

In [3]:
print(train.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [4]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [5]:
train["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

# Inspect the misssing values

In [6]:
total = train.isnull().sum().sort_values(ascending=False)
percent_1 = train.isnull().sum()/train.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0


In [7]:
train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [8]:
test.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

# Filling missing values

In [9]:
#Age


train["Age"] = train["Age"]. fillna(train["Age"].mean())

test["Age"]  = test["Age"] . fillna(test["Age"].mean())


#Fare

test["Fare"] = test ["Fare"]. fillna(test["Fare"].mean())


#Embarked

train["Embarked"].fillna("S", inplace = True)

In [10]:
train.loc[train["Sex"] == "male" , "Sex"] = 0
train.loc[train["Sex"] == "female","Sex"] = 1


test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Sex"] == "female", "Sex"] = 1

In [11]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2


test.loc[test["Embarked"]  == "S", "Embarked"] = 0
test.loc[test["Embarked"]  == "C", "Embarked"] = 1
test.loc[test["Embarked"]  == "Q", "Embarked"] = 2

In [12]:
train["FamSize"] = train["SibSp"] + train["Parch"] + 1
test["FamSize"]  =  test["SibSp"] + test["Parch"]  + 1

In [13]:
train["IsAlone"] = train.FamSize.apply(lambda x: 1 if x == 1 else 0)
test["IsAlone"]  = test.FamSize.apply( lambda x: 1 if x == 1 else 0)

In [14]:
for name in train["Name"]:
    train["Title"] = train["Name"].str.extract("([A-Za-z]+)\.",expand=True)
    
for name in test["Name"]:
    test["Title"] = test["Name"].str.extract("([A-Za-z]+)\.",expand=True)
    
title_replacements = {"Mlle": "Other", "Major": "Other", "Col": "Other", "Sir": "Other", "Don": "Other", "Mme": "Other",
          "Jonkheer": "Other", "Lady": "Other", "Capt": "Other", "Countess": "Other", "Ms": "Other", "Dona": "Other", "Rev": "Other", "Dr": "Other"}

train.replace({"Title": title_replacements}, inplace=True)
test.replace({"Title": title_replacements}, inplace=True)

train.loc[train["Title"] == "Miss", "Title"] = 0
train.loc[train["Title"] == "Mr", "Title"] = 1
train.loc[train["Title"] == "Mrs", "Title"] = 2
train.loc[train["Title"] == "Master", "Title"] = 3
train.loc[train["Title"] == "Other", "Title"] = 4

test.loc[test["Title"] == "Miss", "Title"] = 0
test.loc[test["Title"] == "Mr", "Title"] = 1
test.loc[test["Title"] == "Mrs", "Title"] = 2
test.loc[test["Title"] == "Master", "Title"] = 3
test.loc[test["Title"] == "Other", "Title"] = 4

In [15]:
print(set(train["Title"]))

{0, 1, 2, 3, 4}


In [16]:
features_drop = ['Ticket', 'SibSp', 'Parch', "Name", "Cabin", "Fare", "PassengerId"]

train = train.drop(features_drop, axis=1)

test = test.drop(features_drop, axis=1)

In [17]:
train = pd.get_dummies(train, columns=['Pclass','Sex','Embarked','Title'], 
                       drop_first=False)

test = pd.get_dummies(test, columns=['Pclass','Sex','Embarked','Title'],
                      drop_first=False)

In [18]:
X = train.drop('Survived', axis=1)

y = train['Survived']

X.shape,y.shape

((891, 16), (891,))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state= 1)

In [20]:
clfs = [('LR', LogisticRegression()),
        ('KNN3', KNeighborsClassifier(n_neighbors=3)),
        ('KNN5', KNeighborsClassifier(n_neighbors=5)),
        ('KNN7', KNeighborsClassifier(n_neighbors=7)),
        ('KNN9', KNeighborsClassifier(n_neighbors=9)),
        ('SVM_l', SVC(kernel='linear')),
        ('SVM_r', SVC(kernel='rbf')),
        ('DT2', DecisionTreeClassifier(max_depth=2)),
        ('DT3', DecisionTreeClassifier(max_depth=3))]

In [21]:
for name, clf in clfs:
    clf.fit(X_train, y_train)
    print('{}:\ttrain_acc:{},\ttest_acc:{}'.format(
        name,
        clf.score(X_train, y_train),
        clf.score(X_test, y_test),
        ))

C:\Users\shadm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR:	train_acc:0.8426966292134831,	test_acc:0.8100558659217877
KNN3:	train_acc:0.8904494382022472,	test_acc:0.8044692737430168
KNN5:	train_acc:0.8637640449438202,	test_acc:0.7597765363128491
KNN7:	train_acc:0.8497191011235955,	test_acc:0.7374301675977654
KNN9:	train_acc:0.8384831460674157,	test_acc:0.7262569832402235
SVM_l:	train_acc:0.8384831460674157,	test_acc:0.8044692737430168
SVM_r:	train_acc:0.648876404494382,	test_acc:0.6033519553072626
DT2:	train_acc:0.8019662921348315,	test_acc:0.7374301675977654
DT3:	train_acc:0.8356741573033708,	test_acc:0.7932960893854749


In [22]:
en_clf = VotingClassifier(clfs, n_jobs=-1)
en_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('KNN3',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',...
                              DecisionTreeCl

In [23]:
print(en_clf.score(X_train, y_train))
print(en_clf.score(X_test, y_test))

0.8553370786516854
0.7821229050279329


In [24]:
y_pred_train = en_clf.predict(X_train)

y_pred_test = en_clf.predict(X_test)

In [25]:
pd.DataFrame(confusion_matrix(y_test, y_pred_test),\
             columns= ["Predicted Not Survived", "Predicted Survived"],\
             index = ["Not-Survived", "Survived"])

,Predicted Not Survived,Predicted Survived
Not-Survived,97,9
Survived,30,43


In [26]:
passenger_IDs = pd.read_csv("E:/Kaggle.Challenges/Machine.Learning.Titanic/Dataset/test.csv")[["PassengerId"]].values
preds = en_clf.predict(test.values)
preds

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [27]:
df = {'PassengerId': passenger_IDs.ravel(), 'Survived': preds}
df_predictions = pd.DataFrame(df).set_index(['PassengerId'])
df_predictions.head(10)

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
897,0
898,1
899,0
900,1


In [28]:
df_predictions.to_csv('E:/Kaggle.Challenges/Machine.Learning.Titanic/predictions_Survived.csv')